## LangChain Expression Language(LCEL)

https://python.langchain.com/v0.1/docs/expression_language/why/

### 기본 구조: 프롬프트 + 모델 + 출력 파서


In [77]:
# 필요한 패키지 설치
!pip install -Uq python-dotenv langchain_teddynote langchain_openai langchain langchain-community faiss-cpu

In [78]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv("../content/.env", override=True)

True

In [79]:
#API KEY 저장을 위한 os 라이브러리 호출
import os

os.environ['LANGCHAIN_PROJECT'] = 'LCEL'
print(f"[LANGCHAIN_PROJECT]\n{os.environ['LANGCHAIN_PROJECT']}")

[LANGCHAIN_PROJECT]
LCEL


In [80]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("LCEL")

LangChain/LangSmith API Key가 설정되지 않았습니다. 참고: https://wikidocs.net/250954


### 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿
  - `template`: 템플릿 문자열. 문자열 내에서 중괄호 `{}`는 변수를 나타냄
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의함

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트

In [81]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [82]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [83]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [84]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [85]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    max_tokens=2048,
    temperature=0.1,
)

### [실습] 셰프 프롬프트 템플릿 만들기

1. 변수 `food`에 음식 식재료를 입력받기 
2. `food` 재료로 만들 수 있는 음식 메뉴를 추천해주는 프롬프트 템플릿 객체 `chef_prompt` 생성하기
3. `food` 변수에 재료(값)를 넣어서 프롬프트를 완성해보기

In [90]:
# 실습 코드 작성
# template 정의
template = "{food}로 만들 수 있는 음식 메뉴를 추천해주세요"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
chef_prompt = PromptTemplate.from_template(template)
chef_prompt

prompt = chef_prompt.format(food="토마토")

model.invoke(chef).content

'1. 토마토 파스타: 신선한 토마토와 마늘, 올리브 오일을 활용하여 맛있는 토마토 파스타를 만들어 보세요.\n2. 토마토 스튜: 토마토, 양파, 당근, 감자 등 신선한 채소와 함께 스튜로 조리하여 푸짐하고 건강한 한 끼 식사를 즐겨보세요.\n3. 토마토 수프: 토마토, 양파, 당근, 락토프로 레 시피 츨릭 쑴다!\n4. 토마토 샐러드: 당근, 오이, 파슬리 등 다양한 채소와 함께 싱싱한 토마토를 활용하여 상큼한 샐러드를 만들어 즐겨보세요.\n5. 토마토 리조또: 토마토 소스와 총친 겠다.  디토마토와 치즈, 오레가노를 넣어 맛있는 토마토 리조또를 만들어보세요.'

### Chain 생성

#### LCEL(LangChain Expression Language)


```
chain = prompt | model | output_parser
```

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달


In [100]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI()

chain = prompt | model
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 쉽게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001D1805FD2B0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D1805FD390>, root_client=<openai.OpenAI object at 0x000001D1805D9A90>, root_async_client=<openai.AsyncOpenAI object at 0x000001D1805D9B80>, model_kwargs={}, openai_api_key=SecretStr('**********'))

### [실습] 셰프 챗봇을 위한 openAI 모델 생성

1. 변수 `chef_model` 선언하기
2. 사용할 모델은 `gpt-4o-mini` 지정
3. 창의성을 조절하는 변수는 `0.1` 로 지정
4. 최대 활용할 토큰은 `4096`

In [101]:
# 실습 코드 작성
chef_model = ChatOpenAI(
    model="gpt-4o-mini",
    max_tokens=4096,
    temperature=0.1,
)

### [실습] 셰프 챗봇을 위한 chef_chain 생성

- 프롬프트 템플릿 변수 `chef_prompt` 와 llm 모델을 담은 `chef_model` 모델을 체인(`|`)으로 연결하여 `chef_chain` 변수 생성

In [102]:
# 실습 코드 작성
chef_chain = chef_prompt|chef_model
chef_chain

PromptTemplate(input_variables=['food'], input_types={}, partial_variables={}, template='{food}로 만들 수 있는 음식 메뉴를 추천해주세요')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001D1805FE350>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D1805FD630>, root_client=<openai.OpenAI object at 0x000001D1805D9D60>, root_async_client=<openai.AsyncOpenAI object at 0x000001D1805DA120>, model_name='gpt-4o-mini', temperature=0.1, model_kwargs={}, openai_api_key=SecretStr('**********'), max_tokens=4096)

### invoke() 호출

- python 딕셔너리 형태(키: 값)로 입력값을 전달
- invoke() 함수 호출 시, 입력값을 전달

In [103]:
# input 딕셔너리에 주제 설정
input = {"topic": "인공지능의 학습 방법"}

In [104]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달
# 이를 통해 AI 모델이 생성한 메시지를 반환
chain.invoke(input)

AIMessage(content='인공지능의 학습 방법은 크게 지도 학습, 비지도 학습, 강화 학습으로 나눌 수 있습니다.\n\n1. 지도 학습: 레이블이 달린 데이터를 이용하여 모델을 학습시키는 방법입니다. 학습 데이터와 정답 레이블을 모델에 입력하여 모델이 입력 데이터와 정답 데이터 간의 패턴과 관계를 학습하게 됩니다.\n\n2. 비지도 학습: 레이블이 달려있지 않은 데이터를 이용하여 모델을 학습시키는 방법입니다. 데이터 간의 숨겨진 패턴과 관계를 모델이 스스로 학습하게 됩니다. 주로 군집화, 차원 축소, 이상치 탐지와 같은 작업에 사용됩니다.\n\n3. 강화 학습: 보상 시스템을 통해 모델이 주어진 환경에서 어떤 행동을 취해야 하는지 학습하는 방법입니다. 모델은 환경과 상호작용하며 보상을 최대화하기 위해 최적의 행동을 학습하게 됩니다.\n\n이러한 학습 방법들을 조합하여 다양한 인공지능 모델을 학습시킬 수 있으며, 이를 통해 문제 해결이나 예측에 활용됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 402, 'prompt_tokens': 30, 'total_tokens': 432, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CJrekpoDCJ2pxLKlAnyQ1T2VctbQF', 'service_tier': 'default', 'finish_reason': 'stop'

In [97]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능의 학습 방법은 크게 지도학습, 비지도학습, 강화학습으로 나눌 수 있습니다.

1. 지도학습: 가장 일반적인 학습 방법으로, 레이블이 달린 데이터를 사용하여 모델을 학습시키는 방법입니다. 즉, 입력 데이터와 정답 데이터(레이블)를 모델에 제공하여 모델이 입력 데이터와 정답 데이터 간의 패턴을 학습하게 됩니다. 대표적으로 분류나 회귀 문제에 사용됩니다.

2. 비지도학습: 레이블이 없는 데이터를 사용하여 모델을 학습시키는 방법으로, 데이터의 내부 구조나 패턴을 찾아내는 데 사용됩니다. 클러스터링이나 차원 축소 같은 작업에 많이 사용됩니다.

3. 강화학습: 에이전트가 환경과 상호작용하며 보상을 최대화하는 방향으로 학습하는 방법입니다. 에이전트는 행동을 선택하고 환경으로부터 보상을 받아 보상을 최대화하는 행동을 선택하는 방향으로 학습하게 됩니다. 주로 게임이나 로봇 제어와 같은 분야에서 사용됩니다.

이러한 학습 방법을 통해 인공지능 모델은 데이터를 분석하고 패턴을 찾아내어 문제를 해결하거나 예측하는 능력을 향상시킬 수 있습니다.

### 출력파서(Output Parser)


In [98]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [105]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser
chain

PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='{topic} 에 대해 쉽게 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001D1805FD2B0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D1805FD390>, root_client=<openai.OpenAI object at 0x000001D1805D9A90>, root_async_client=<openai.AsyncOpenAI object at 0x000001D1805D9B80>, model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser()

In [106]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능의 학습 원리"}
chain.invoke(input)

'인공지능의 학습 원리는 데이터를 입력 받고 이를 분석하여 패턴이나 규칙을 학습하는 것입니다. 이러한 데이터는 사람들이 레이블을 달아주는 지도학습이나 스스로 패턴을 찾아내는 비지도학습 방식으로 학습할 수 있습니다. 또한 강화학습을 통해 경험을 토대로 보상을 최대화하는 방법을 학습하기도 합니다. 이렇게 학습한 인공지능은 새로운 데이터가 주어졌을 때 그것을 분석하여 판단하거나 행동할 수 있게 됩니다.요약하자면, 인공지능은 데이터를 입력 받고 이를 분석하여 패턴이나 규칙을 학습하여 새로운 상황에서 판단하거나 행동할 수 있게 됩니다.'

In [107]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능은 기계가 데이터를 분석하고 학습하여 스스로 패턴을 발견하고 결정을 내리는 기술입니다. 

인공지능의 학습 원리는 크게 지도학습, 비지도학습, 강화학습으로 나뉩니다. 

1. 지도학습: 기계가 입력된 데이터와 그에 대한 정답을 함께 학습하여 패턴을 파악하고 정확한 결과를 도출하는 방법입니다. 학습 데이터와 정답 데이터를 기반으로 모델을 학습시켜 예측을 수행합니다. 

2. 비지도학습: 기계가 주어진 데이터에서 패턴을 스스로 찾아내어 학습하는 방법입니다. 정답 데이터 없이 입력된 데이터만을 기반으로 모델을 학습시켜 유용한 정보를 추출하거나 클러스터링을 수행합니다.

3. 강화학습: 기계가 특정 환경에서 행동을 한 후 그 결과에 대한 보상이나 벌점을 받아서 학습하는 방법입니다. 보상을 최대화하고 벌점을 최소화하는 방향으로 행동을 결정하여 학습하며, 주로 게임이나 로봇 제어 등에 사용됩니다.

이러한 학습 원리를 바탕으로 인공지능 기술은 계속 발전하며 다양한 분야에 적용되고 있습니다.

### [실습] 최종! 셰프 챗봇을 실행해보기

- 마지막 StrOutputParser()까지 추가하여 `chef_chain` 완성하기
- 변수 `food_input`에 딕셔너리 형태로 재료 넣어 `chef_chain` 실행해보기
- Ex) 두부, 김치, 당근 ...

In [116]:
# 실습
food_input = {'food' : "오이"}
chef_chain = chef_prompt|chef_model|output_parser
# chef_chain.invoke(food_input)

stream_response(chef_chain.stream(food_input))

오이를 활용한 다양한 음식 메뉴를 추천해드릴게요!

1. **오이무침**: 오이를 채 썰어 고추가루, 다진 마늘, 간장, 식초, 설탕 등을 넣고 무쳐서 상큼한 반찬으로 즐길 수 있습니다.

2. **오이 샐러드**: 오이를 슬라이스하여 토마토, 양파, 드레싱(올리브유, 레몬즙, 소금 등)과 함께 섞어 신선한 샐러드를 만들어 보세요.

3. **오이김치**: 오이를 소금에 절인 후, 고춧가루, 마늘, 생강, 새우젓 등을 넣고 버무려서 간단한 김치를 만들어 보세요.

4. **오이 스무디**: 오이를 믹서에 갈아 요거트, 바나나, 꿀과 함께 섞어 건강한 스무디를 만들어 보세요.

5. **오이롤**: 오이를 얇게 썰어 햄이나 크림치즈, 아보카도 등을 넣고 말아서 간단한 롤을 만들어 보세요.

6. **오이냉국**: 오이를 얇게 썰어 시원한 육수(멸치육수나 다시마육수)에 넣고, 간장과 식초로 간을 맞춰서 여름철에 좋은 냉국을 만들어 보세요.

7. **오이 피클**: 오이를 소금에 절인 후, 식초, 설탕, 향신료를 넣고 절여서 아삭한 피클을 만들어 보세요.

이 외에도 오이는 다양한 요리에 활용할 수 있으니, 창의적으로 조리해 보세요!

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 가능
- `model_name` 역시 변경하여 테스트가 가능

In [117]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화
output_parser = StrOutputParser()

In [118]:
# 체인을 구성
chain = prompt | model | output_parser

In [119]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
  - Waiter: Good evening! Welcome to our restaurant. How many people are in your party?
  - You: Good evening! It's just me.
  - Waiter: Great! Here’s the menu. Can I start you off with something to drink?
  - You: Yes, I’d like a glass of water, please.
  - Waiter: Sure! Are you ready to order your food?
  - You: Yes, I’d like the grilled chicken salad, please.
  - Waiter: Excellent choice! Would you like any dressing with that?
  - You: Yes, please. I’ll have the balsamic vinaigrette.
  - Waiter: Perfect! I’ll get that started for you.

- 한글 해석:
  - 웨이터: 좋은 저녁입니다! 저희 식당에 오신 것을 환영합니다. 몇 분이신가요?
  - 당신: 좋은 저녁입니다! 저 혼자입니다.
  - 웨이터: 좋습니다! 여기 메뉴입니다. 음료는 뭘 드릴까요?
  - 당신: 네, 물 한 잔 주세요.
  - 웨이터: 알겠습니다! 음식 주문할 준비 되셨나요?
  - 당신: 네, 구운 치킨 샐러드로 주세요.
  - 웨이터: 훌륭한 선택입니다! 드레싱은 무엇을 원하시나요?
  - 당신: 네, 발사믹 비네그레트를 주세요.
  - 웨이터: 완벽합니다! 바로 준비하겠습니다.


In [ ]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

In [120]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:
  - Customer: Hi there! I’d like to order a pizza, please.
  - Pizza Shop: Sure! What size would you like?
  - Customer: I’ll have a large one, please. 
  - Pizza Shop: Great choice! What toppings do you want?
  - Customer: Can I get pepperoni, mushrooms, and extra cheese?
  - Pizza Shop: Absolutely! Would you like any drinks or sides with that?
  - Customer: Yes, I’d like a bottle of soda, please.
  - Pizza Shop: Perfect! Your total comes to $25. How would you like to pay?
  - Customer: I’ll pay with my credit card.
  - Pizza Shop: Good. Can I have your address for delivery?
  - Customer: Sure! It’s 123 Main Street, apartment 4B.
  - Pizza Shop: Thank you! Your pizza will be there in about 30 minutes.
  - Customer: Thank you!

- 한글 해석:
  - 고객: 안녕하세요! 피자 하나 주문하고 싶습니다.
  - 피자 가게: 네, 어떤 사이즈로 드릴까요?
  - 고객: 큰 사이즈로 주세요.
  - 피자 가게: 좋은 선택입니다! 어떤 토핑을 원하시나요?
  - 고객: 페퍼로니, 버섯, 그리고 치즈 추가로 부탁드려도 될까요?
  - 피자 가게: 물론이죠! 음료수나 사이드 메뉴는 필요하신가요?
  - 고객: 네, 탄산음료 한 병 주세요.
  - 피자 가게: 알겠습니다! 총 금액은 25

### [실습] 템플릿을 변경하여 나만의 여행 가이드 챗봇 만들기

- 위의 프롬프트를 아래 주제에 맞게 **변경** 해보기
1. 페르소나: 10년차 여행 가이드
2. 3일간 가성비 여행 계획을 세워주는 챗봇 생성
3. `{question}` 에는 여행갈 나라와 도시를 사용자에게 입력받음
4. `answer` 변수를 출력하여 챗봇의 답변 결과 확인
5. `Langsmith` 에 접속하여 실행 내용 확인

In [125]:
tour_guide_template = """
당신은 10년차 여행 가이드 입니다. 주어진 지역에 맞는 3일간 가성비 여행 계획을 세워주세요
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황
{question}

#FORMAT:
1일차
2일차
3일차
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성
prompt = PromptTemplate.from_template(tour_guide_template)

# ChatOpenAI 챗모델을 초기화
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화
output_parser = StrOutputParser()

# 체인을 구성
chain = prompt | model | output_parser

In [126]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 대한민국 서울을 여행하고 싶습니다."})
# 스트리밍 출력
stream_response(answer)

# 서울 3일간 가성비 여행 계획

## 1일차
- **오전**: 
  - 경복궁 탐방 (무료 또는 3,000원)
    - 고궁의 아름다움을 감상하고, 한복 체험(대여 약 10,000원)도 추천.
- **점심**: 
  - 인사동 지역의 전통 한식당(1인 10,000원 내외).
- **오후**: 
  - 북촌 한옥마을 산책 (무료).
    - 전통 한옥을 구경하며 사진 촬영.
- **저녁**: 
  - 홍대의 길거리 음식 탐방 (1인 5,000원 내외).
    - 떡볶이, 핫바, 붕어빵 등 다양하게 즐기기.

## 2일차
- **오전**: 
  - 남산타워(서울타워) 방문 (입장 비용 약 10,000원, 케이블카 비용 별도).
    - 서울의 전경 감상.
- **점심**: 
  - 명동의 음식 골목에서 미식 탐방 (1인 8,000원 내외).
- **오후**: 
  - 서울역사박물관 탐방 (무료).
    - 서울의 역사와 문화를 배우며 흥미로운 전시 관람.
- **저녁**: 
  - 광장시장에서 다양한 길거리 음식 체험 (1인 5,000원 내외).

## 3일차
- **오전**: 
  - 창덕궁 탐방 (입장료 3,000원).
    - 유네스코 세계유산으로 지정된 아름다운 정원 관람.
- **점심**: 
  - 종로1가의 백숙집에서 간단한 식사 (1인 10,000원 내외).
- **오후**: 
  - 청계천 산책 (무료).
    - 도심 속에서 자연을 느끼며 여유로운 시간을 보냄.
- **저녁**: 
  - 이태원의 다양한 세계 음식을 저렴하게 즐길 수 있는 레스토랑 방문 (1인 12,000원 내외).

이 계획은 서울의 주요 명소와 지역 문화를 경험할 수 있으며, 가성비를 고려하여 식비와 들어가는 비용을 최적화했습니다. 즐거운 여행 되세요!

In [124]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 이탈리아 로마를 여행하고 싶습니다."})
# 스트리밍 출력
stream_response(answer)

- 추천 장소: 콜로세움 (Colosseo)  
  로마의 상징적인 유적지로, 고대 로마의 역사와 건축물의 웅장함을 체험할 수 있습니다. 야경이 특히 아름다우니, 저녁 시간에 방문해보세요.

- 추천 음식: 카르보나라 (Carbonara)  
  로마의 전통 파스타 중 하나로, 크림 대신 계란과 치즈, 베이컨이 어우러져 깊은 맛을 자아냅니다. 현지 식당에서 freshly made(신선하게 만든) 카르보나라를 꼭 맛보세요!